### 模擬試験でのポイント

[**Amazon EC2**]
- backupを取得するには、**EC2インスタンスを停止し、各EC2インスタンスボリュームのスナップショットを個別に作成する。** スナップショットが完了したら、EC2インスタンスを起動し、関連するすべてのボリュームが再マウントされていることを確認する

[**AWS Organizations**]<br>
- 複数アカウントをメンバーアカウントとしてグループ化した**OUに対して、サービスコントロールポリシー(SCP)を設定する**ことで、複数アカウント内で設定で切るAWSリソースのアクション権限範囲を管理することができます。
- SCP内で許可した範囲においてIAMポリシーによるアクセス権限付与が実行できるアカウント
  - (イメージ)IAM(EC2・ECSへの許可)、SCP(EC2、RDS・ECSの許可)=EC2・ECSの許可のみ
  - **ユーザーに対してアクセス許可権限を付与しません。**

[**AWS Storage Gateway**]<br>
- オンプレミスアプリケーションによる AWS クラウドストレージのシームレス（違和感なく統合されている）な使用を可能にするハイブリッドストレージサービス。バックアップ、アーカイブ、災害対策、クラウドデータ処理、ストレージの階層化、および移行を行うことができます。
- **保管型ボリューム**を使用すると、プライマリデータをローカルに保存する一方で、そのデータを非同期に AWS にバックアップします。
- 上記は「Amazon Elastic Block Store (Amazon EBS) スナップショットとして Amazon S3 に非同期でバックアップ」される
- キャッシュ型ボリュームを使用することで、頻繁にアクセスされるデータはローカルのストレージゲートウェイに保持しながら、Amazon S3 をプライマリデータストレージとして使用
- つまり保管型が「AWSクラウドにストレージを拡張するケース」で用いられる

[**S3**]<br>
- S3バケットのオブジェクトロックのコンプライアンスモードでは、指定した保持期間中にS3バケット内のオブジェクトに対してデータ変更や削除ができなくなります。
- コンプライアンスモードでは、AWS アカウント の root ユーザーを含め、ユーザーが、保護されたオブジェクトのバージョンを上書きまたは削除することはできません
- ガバナンスモードでは、必要に応じて一部のユーザーにリテンション設定の変更、またはオブジェクトの削除を許可することができます。
- (注意点)**オブジェクトロックの有効化は、S3バケットを作成する際にしか実施できません。**
- (注意点)**バケットポリシーにWAFを設定することができない。WAFはCloudFrontに設定する必要があります。**
- 「データ量を抑えるために保存後に1ヶ月以上すぎた場合は対象ビデオデータを削除すること」の要件をかなえるには**S3 Standard IAにビデオを保存してライフサイクルポリシーで1か月後に削除する。**
- Amazon S3バケットの**GetObject権限を付与するAssumeRoleを実施するIAMロールを作成して、特定のユーザーにのみ権限を委譲することで、クロスアカウントアクセスを可能にします。**
- **AsumeRoleを利用することで IAM ロールに設定された権限を移譲することができます。**
- ドキュメント管理システムでは頻繁にドキュメントの変更が発生するため、**Amazon S3バケットのバージョニングを有効化して、適切なバージョン管理を実行**

[**インターネットゲートウェイとルーティング**]<br>
- パブリックサブネットとしてインターネットからのアクセスを許可するためにはインターネットゲートウェイが必要です。設定としては、このEC2インスタンスにはインターネットから自由にアクセスするWEBサイトを構築する予定であるため、インターネットゲートウェイに対して0.0.0.0/0でフルオープンを設定することが求められる(**Destination：0.0.0.0/0  - >Target：インターネットゲートウェイ**)

[**Auto Scalingグループ**] <br>
- 現在のスケーリング設定では短期間にスケールアウトとスケールインを繰り返しており、問題となっています。この状況を改善するために必要となるスケーリング設定は・・
  - **Auto Scalingグループのクールダウンタイム値を最適化する。**
  - **Auto Scalingのスケールアウト時のポリシーをステップスケーリングポリシーにすることで、インスタンス増減をより詳細に制御する。**

[**Amazon CloudWatch**]<br>
- AWSのリソース（EC2、RDS、Lambdaなど）の監視や、カスタムメトリクスの収集・アラート設定を行うサービスです。CloudWatchを使ってシステムの状態を監視し、**Auto Scalingのトリガーとして利用することができます。**
- Auto Scalingグループを作成した後、CloudWatchアラームを設定して、スケーリングのトリガーとして使用できます。

[**Amazon EFS**]<br>
- EFS のライフサイクル管理では「費用対効果の高いファイルストレージを自動的に管理します。有効にすると、ライフサイクル管理は 30 日間アクセスされなかったファイルを EFS IA ストレージクラスに自動的に移行します。ライフサイクル管理によってファイルが IA ストレージクラスに移動されると、**無期限にデータが残ります。削除をするわけではない**」

[**Amazon RDS DB**]<br>
- Amazon RDS DBインスタンスはリードレプリカを追加して、アプリケーションの読み込み負荷を分散することができる、これは非同期レプリケーションです。
- Optimesd Writeが利用できるMySQLバージョンに変更して、Optimesd Writeを適用して書き込み処理を高速化することも可能。Amazon RDSのOptimesd Writeは書き込み処理を最大50％向上できるRDSの機能
- マルチAZ構成でRDSマスターともう一つのAZへはRDAスレーブを配置して同期レプリケーションによる自動フェイルオーバー機能も追加可能
- Amazon RDSをAuroraサーバレス MySQLに変更することでピーク時にオートスケーリングはされますが、Auroraサーバレス はピーク以外の通常時においても一定のデータベース利用が発生しない、不規則なデータベース利用が発生する際に向いているDBタイプ
- (注意点)クエリ処理には向いていますが、**ログ解析には向いていません**。
- （注意点）**RDSのAuto Scalingはデータ容量を増加させるだけ**つまり読み取りリクエストの増加等には関係ない。読み取り専用のリードレプリカの用意や単純にコストが高くなりますが、より高性能なインスタンスタイプに変更してスケールアップすることも最適な対応

[**Amazon EBS**]<br>
- EBSのスループット最適化HDD
  - 高いスループットを必要とするアクセス頻度の高いワークロード向けの低コストの HDD ボリューム.プロビジョンドIOPS SSDよりも低コストで実現  
- EBSの汎用SSD
   - 幅広いトランザクションワークロードに対応できる価格とパフォーマンスのバランスが取れた汎用 SSD ボリューム
- EBSのプロビジョンドIOPS SSD
   - イテンシーの影響が大きいトランザクションワークロード向けに設計された極めてパフォーマンスの高い SSD ボリューム。コストを度外視して、最も高いスループット性能を求められる際には正解
- EBSのコールドHDD
   - アクセス頻度の低いワークロード向けに設計された極めて低コストの HDD ボリューム
 
[**AWS Backup**]<br>
- **様々なバックアップ処理を一元管理する仕組み**
- **AWS BackupではAMIやスナップショットは別リージョンにコピーすることができます。**
- 具体的には「AWS Backupを利用して、EBSのバックアップを取得して、対象EC2インスタンスのダウンに対して、新インスタンスを立上げてEBSボリュームを復元する。」ことで東京リージョンに障害が発生した場合に備えて、障害回復ソリューションを準備することが可能
- こちらの対応も必須「AWS Backupを利用して、定期的にEC2インスタンスからAMIを取得して、別のリージョンにコピーするようにバックアッププランを作成する。」
- 
[**DynamoDB**]<br>
- DynamoDBをオンデマンドに変更することで、一時的な負荷に対してキャパシティをスケーリングして高負荷に対応することが可能<br>
- (注意)*DynamoDBをマルチAZ構成で起動する機能はない*  
- オートスケーリングを利用したり、Amazon SQSを構成した方がコスト最適に対応。ただし「DynamoDBのオンデマンドモードを選択すると、DynamoDB は、前に到達したトラフィックレベルまで拡張または縮小して、ワークロードを即座に受け入れることができるようにします。したがって、**このモードにはAuto Scalingは必要ありません**」
- アプリケーションの構成を変更することなく、DynamoDBのパフォーマンス効率を向上させるには、**Amazon DynamoDB Accelrator（DAX）を利用して、DynamoDBテーブルにキャッシュレイヤーを追加**して高速処理を実施
- **DynamoDBストリームでは、テーブル内の項目が変更された際の変更内容をストリームして通知する機能**

[**Amazon Redshift**]<br>
- **AWSのフルマネージドなデータウェアハウスサービス**で、**大規模なデータ分析を高速かつスケーラブルに行うことができます。** Redshiftは、特に**データの集計、分析、レポーティングに最適化されており、ビッグデータ環境での使用に向いています。**
- **小規模利用からペタバイト単位の構造化データまで、複雑な分析クエリを実行でき、スケールアウトも容易に行うことができます。** Amazon Redshift データウェアハウスは、ノードと呼ばれるコンピューティングリソースのコレクションであり、これらはクラスターと呼ばれるグループを構成します。各クラスターは、1 つの Amazon Redshift エンジンを実行し、1 つ以上のデータベースを含みます。
- RDSのマルチAZ配置と同様に複数AZに展開が可能
- **大規模なトランザクションデータやログデータを集約して、高速にクエリを実行し、ビジネスインテリジェンスツール（例: Tableau、Power BI）で視覚的なレポートやダッシュボードを作成できます。**

[**CloudFront**]<br>
- CloudFrontは、コンテンツ配信ネットワーク（CDN）で、エンドユーザーに近いエッジロケーションからコンテンツを提供します。CloudFrontを使用することで、ウェブアプリケーションのレスポンス速度を向上させ、ユーザー体験を改善
- Amazon CloudFrontのフィールドレベル暗号化を実施して、アップロード時のデータの暗号化を実施することで、ユーザーによるデータアップロード時の暗号化に対するセキュリティレイヤーを追加することができます。さらに、フィールドレベル暗号化では、特定のデータに特定のアプリケーションのみがアクセスできるように制限することができる
- フィールドレベル暗号化により、ユーザーが機密情報をウェブサーバーに安全にアップロード
- (注意点)**AWS KMSを適用して、アップロード時の暗号化を実施することはできません。**
- CloudFrontディストリビューションを作成し、**オリジン（元となるサーバ）としてALBのDNS名を設定**します。これにより、CloudFrontはALBを通じてEC2インスタンスにリクエストをルーティングします。
- 必要に応じて、キャッシュ設定や、SSL証明書（HTTPS通信を使用する場合）の設定を行います。
- **エッジロケーションによるファイル圧縮処理を実施する。ことで経費削減が可能**
   - ビューワーがリクエストヘッダーに Accept-Encoding: gzip を含めるようのリクエストした場合は、CloudFront が自動的に特定のタイプのファイルを圧縮
   - CloudFront は各エッジロケーションでファイルを圧縮
- **CloudFrontによるキャッシ保持期間を長くすることで経費削減が可能**
- （注意点）Amazon EC2インスタンスとALBを利用して動画配信アプリケーションを構成している場合は、**Amazon CloudFrontディストリビューションをEC2インスタンスではなくALBに設定することが必要**

[**AWS Global Accelerator**]<br>
- パブリックアプリケーションの可用性、パフォーマンス、およびセキュリティの向上させるためのネットワークサービス
- エッジでパケットをプロキシして最適なエンドポイントグループにルーティングするかを決めることで、TCP または UDP を利用した幅広いアプリケーションのパフォーマンスを向上

[**NATゲートウェイ**]
- NATインスタンスからNATゲートウェイに移行する決定は、**主に高帯域幅のトラフィック要求に対応するため** これを実現するには「NATインスタンスからNATゲートウェイに移行して、**NATゲートウェイをパブリックサブネットに設定**する。」

[**VPCエンドポイント**]
- VPCエンドポイントはインターネットゲートウェイ、NATデバイス、VPN接続、またはAWS Direct Connect接続を必要とせずに、**VPCをPrivateLinkのサポート対象のAWSサービスとVPCエンドポイントサービスに接続**

[**CloudFormationテンプレート**]<br>
- **Egress: true で「アウトバウンド許可」**

[**ネットワークACL**]<br>
- Network Access Control Listの略で、AWSにおけるVPC (Virtual Private Cloud) 内で使用される、**サブネットレベルのセキュリティ機能**の一つ
- ネットワークACLは**ステートレス**です。つまり、リクエストのレスポンスに関する情報を保持せず、インバウンドとアウトバウンドのトラフィックを独立して扱います
- **デフォルトのネットワークACLは、すべてのインバウンドおよびアウトバウンドトラフィックを許可**
- （注意点）トラフィックやプロトコルタイプに応じて、トラフィック制御に利用される機能であり、**暗号化を達成する方法ではありません。**
  
[**SQS**]<br>
- SQSとAuto Scalingを構成する際はCloudWatchメトリックスに基づいてキューの処理量に応じたスケーリングを設定
- 完全マネージド型のメッセージキューイングサービスで、マイクロサービス、分散システム、およびサーバーレスアプリケーションの切り離しとスケーリングが可能
- 負荷分散が可能
- 送信順序を守られることが要件の場合は、「**FIFOキューを利用**」
- **メッセージグループID**を利用することで同じグループのメッセージはまとめて送信されるようにFIFOキューを調整することができます。

[**Amazon SNS**]<br>
- イベント通知を起点としたトリガーとして利用し、実処理はSQSに連携してポーリングするというのが標準的なAWSでのアーキテクチャ構成
- コンポーネント処理間の連携を確実に実行するには並列処理による負荷分散が可能なAmazon SQSが最適
- (注意点)　Amazon SNSにおいてFIFOトピックを利用して、メッセージの送信順序を守ることができますが、**グループ化する仕組みはありません。**


[**Amazon Kinesis Data Streams**]<br>
- デバイスなどから送られる大量のストリーミングデータをリアルタイムに収集、処理して別のサービスに送信するサービス
- **ログ解析には向いていません。**

[**Amazon Kinesis Data Kinesis Firehose**]<br>
- 大量のストリーミングデータを取得して、Lambda関数によってETL処理をしつつ、RedShiftやS3などのストレージやデータベースに配信するサービス
- (注意点)**画像データをAmazon S3バケットに配信することはできません。**

[**Amazon SWF**]<br>
クラウドのワークフロー管理アプリケーションで、複数マシン間でアプリケーションを連携させるためのツールを開発者に提供

[**Amazon EMR**]<br>
- Apache Sparkなどのソフトウェアを構成して、デバイスなどから送られる大量のストリーミングデータをリアルタイムに収集、分析するサービス
- Amazon EMRは、動的にスケーラブルなAmazon EC2インスタンス全体で膨大な量のデータ処理を簡単、高速、そして費用対効果の高いものにするマネージド**Hadoopフレームワーク**を提供します.
- Webインデックス作成、データマイニング、ログファイル分析、機械学習にむいてる

[**Amazon Inspector**]<br>
- 自動化されたセキュリティ評価サービスで、AWS にデプロイしたアプリケーションのセキュリティとコンプライアンスを向上させることができます。EC2インスタンスにエージェントをインストールして、そこにホストされたアプリケーションの意図しないネットワークエクスポージャーなどの脆弱性を検知

[**Amazon GuardDuty**]<br>
- Amazon S3 ログ、CloudTrail 管理イベントログ、DNS ログ、Amazon EBS ボリュームデータ、Kubernetes Audit Logs、Amazon VPC フローログ、RDS ログインアクティビティの AWS CloudTrail データイベントなどのデータソースを分析して、DDoS攻撃以外の様々なネットワークへの不正アクセスを自動的に検知

[**AWS Compute Optimize**]<br>
- EC2インスタンスタイプ、Amazon Elastic Block Store (EBS) ボリューム、AWS Lambda 関数の 3 種類の AWS リソースの過剰プロビジョニングや過小プロビジョニングなどの利用状況に基づいて、コスト最適化のアドバイスを実施

[**Amazon ALB**]<br>
- ALBは**レイヤー7の負荷分散を行い、アプリケーションレベルでのルーティングや、ヘルスチェックをサポート**します。


[**AWS Fargate**] <br>
- AWS Fargateは、Amazon Web Services（AWS）が提供する**サーバーレスコンテナサービス**で、コンテナの管理やインフラのプロビジョニングを自動化することができます。Fargateを使用すると、コンテナの実行に必要なサーバーやインスタンスを管理することなく、アプリケーションを展開できる
- Fargateは、コンテナの実行に必要なサーバーリソースを自動で割り当て、スケールします。**開発者はコンテナの実行に必要なリソース（CPU、メモリなど）を指定するだけで、インフラの設定や管理を行う必要はありません。**
- Fargateは、コンテナの負荷に応じてリソースをスケールできます。
- AWS Fargateは、Amazon ECS（Elastic Container Service） や Amazon EKS（Elastic Kubernetes Service） で動作します。これにより、Fargateを使ってコンテナ化されたアプリケーションをECSやEKSの管理下で簡単にデプロイできます。
- Amazon ECSクラスターをAWS Fargateを起動タイプを利用してプロビジョニングすることで要件を達成することができます。**コンテナをFargateタスクにデプロイすることで、コンピューティングリソースをサーバレスに起動・スケールすることができるため、コンテナオーケストレーションの管理オーバーヘッドをなるべく抑制しつつ、タスクを利用したマイクロサービスを構成することができます。**

[**Amazon ECS**]<br>
-Amazon ECS は、コンテナ化されたアプリケーションを簡単にデプロイ、管理、スケーリングすることができるフルマネージド型のコンテナオーケストレーションサービスです。**Amazon ECS はコンテナ構成時に、EC2インスタンスをコンテナに利用するEC2起動モードと、サーバレスコンピューティングサービスであるFargate起動モードを選択することができます。**




### 知見
[**CloudFrontとHTTP領域**]<br>
CloudFrontを利用しグローバルにコンテンツを配信することを考えた場合「言語処理」
を例えば以下のようにHTTP領域にて展開する方法もある。パラメータをこれで送ることが可能
- http://pintor.cloudfront.net/index?language=jp
- http://pintor.cloudfront.net/index?language=en<br>
この場合は「クエリ文字列パラメーター値に基づくキャッシュ設定を行うことでHTTPフィールド内の言語表示に基づいて配信方法を切り替えること」で実現可能

(補足)<br>
クエリ文字列（URLパラメーター）とは、サーバーに情報を送るためにURLの末尾につけ足す文字列（変数）のことです。「?」をURLの末尾につけ、その次に「パラメーター＝値」をつけ、複数のパラメーターをつけたい場合は「&」を使用します。この形式で、サーバーに送信したいデータをURLにつけ加えることが可能

[**SQSによるマイクロサービス化されたコンポーネントとの連携処理**]<br>
マイクロサービス化されたコンポーネント間の処理を連携する際は、Amazon SQSを利用したポーリング処理が最適となります。
例えば：Amazon ECSを活用してマイクロサービス型のアプリケーションを構築しています。 このアプリケーションのアーキテクチャでは、マイクロサービス１とマイクロサービス２に分割し、マイクロサービス１の処理後にマイクロサービス２に処理が引き渡されます。
]

[**IoTセンサーに基づいたデータ分析**]<br>
上記のようなケースではまさに「Amazon Kinesis Data Streams」を活用すべき。Amazon Kinesis Data Firehose経由でAmazon S3バケットに送信。<br>
注意点として「デフォルトでは、データレコードがAmazon Kinesis Data Streamsに投入されてから24時間以内までアクセスできるように設定されている。」

[**ELBから結合されたログファイルを分析できるようにアプリケーションを構築**]<br>
Amazon S3に対してELBのログファイルを収集し、Amazon EMRでログ解析を実施する方法が考えられる。
ログファイルの収集場所としてはAmazon S3が適切。このログファイル分析には大量データ解析が可能なパフォーマンスが必要

[**エッジ処理とは？**]<br>
「エッジ処理」とは、CloudFrontエッジロケーションで実行される処理のことを指します。エッジロケーションでは、リクエストの処理が行われる際、以下のような処理が実施されることがあります：
- コンテンツキャッシュ: CloudFrontエッジサーバーが、ウェブサイトの静的コンテンツ（HTML、画像、動画、CSS、JavaScriptなど）をキャッシュし、リクエストがあるたびに高速に返します。これにより、リクエスト元のサーバー（オリジンサーバー）への負荷を軽減し、応答速度を向上させます。
- リクエスト/レスポンスのカスタマイズ: CloudFrontは、Lambda@Edgeという機能を使って、リクエストやレスポンスに対してカスタマイズを行うこともできます。これにより、リダイレクトや認証、リクエストヘッダーの変更、レスポンス内容の変更など、エッジでの処理を行えます。
- 動的コンテンツの処理: 一部の動的コンテンツ（例えば、ユーザーごとに異なるデータを提供するAPIレスポンスなど）も、エッジサーバーで処理され、オリジンサーバーへのアクセスが減少します。これにより、コンテンツ配信が迅速になります。

[**データベース障害を抑止するソリューションとコスト最小限で１つのデータセンターに依存した構成**]
- **データを同期的にレプリケートするようにマルチAZ配置を有効にしたAmazon RDS DBインスタンスを構成**する。

[**Amazon S3に保存されているデータにアクセスする必要がありますが、 トラフィックはインターネットを通過できない場合は？**]
- **VPCのプライベートサブネットにインターフェースエンドポイントを設定**して、IPアドレス経由でアクセスする。

[**EC2インスタンス側の暗号化はどうやるの？**]
- 利用している**EBSボリュームの有効化**を実施する必要があります。Amazon EBSボリュームは**AWS KMSと連携**して暗号化キーを利用して、容易にデータを暗号化することができます。

[**Amazon S3の保存データを保護するには？**]
- サーバーサイド暗号化を使用つまり、**オブジェクトをデータセンターのディスクに保存する前にAmazon S3で暗号化**し、オブジェクトを**ダウンロードするときに復号化**するように要求します。S3のデフォルト暗号化を有効化する際に、**SSE-S3**と**SSE-KMS**のどちらかの暗号化形式を選択
- クライアントサイド暗号化を使用つまり、データをクライアントサイドで暗号化し、暗号化されたデータをAmazon S3にアップロードできます。この場合は、暗号化プロセス、暗号化キー、および関連ツールを管理

[**SSL/TLS証明書を用いた基礎**]
- セキュリティ要件として、保管中およびAWSに転送中のデータを保護するための方策の場合は、SL/TLS証明書を利用して、HTTPSプロトコルを利用した通信を実施しています。を使用するロードバランサ（SSLオフロードとも呼ばれる）を作成できます。この機能により、ロードバランサーとHTTPSセッションを開始するクライアント間、およびロードバランサーとEC2インスタンス間の接続のトラフィック暗号化が有効

[**S3バケットのコンテンツをCloudFrontディストリビューションから配信する際に、S3 オブジェクトURLに直接アクセスできないようにしてから、WAFを適用することが求められるケースでは？**]
- Amazon CloudFront側でオリジンアクセスアイデンティティ（OAI）またはオリジンアクセスコントロール (OAC) を作成し、そのOAIまたはOACにのみ読み取りアクセスを許可するようにS3バケットポリシーを設定
- こうすることで、OAIまたはOACのみがS3バケットにアクセスできる構成となり、CloudFrontはOAIまたはOACを介してS3バケットのコンテンツにアクセスして配信処理を行います。


[**既存のEC2インスタンスからAmazon RDS DBインスタンスへのアクセスがIPアドレスを経由して設定されています。Auto Scalingグループから新しいインスタンスが起動する際に、データ処理時にエラーが発生どうする？**]
- Amazon RDSのセキュリティグループにおいて、EC2インスタンスに設定されたセキュリティグループIDをターゲットにして、3306ポート番号のアクセス許可を設定する。Auto Scalingグループで選択する起動テンプレート設定で同じセキュリティグループを利用する。

[**DB構成を変更して、DBインスタンスの高可用性、読み取りワークロードの容量の増加、および書き込みレイテンシーの低減を最大限に達成するには？**]
- 既存のデータベースを**Amazon RDS マルチAZクラスター配置に変更する。** 書き込みリクエストはプライマリのDBインスタンスで処理して、読み込みリクエストはリードレプリカで処理する。





  